# Decorators For Fun And Profit
just read [The ultimate guide](https://realpython.com/primer-on-python-decorators/) if you've got the time, there's also [this](https://wiki.python.org/moin/PythonDecoratorLibrary) wiki entry. check out the [decorator](https://github.com/micheles/decorator/blob/master/docs/documentation.md) library, it could be usefull.

## Function Decorators
This is the most useful and popular decorators. let's start with the simpler use case

In [ ]:
import functools

def wrapper(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print('inner')
        return func(*args, **kwargs)
    return inner

@wrapper
def f():
    print('hi')
f()

Here is a way to look at it. when the program get interpeted, the following psuedo-command is executed: `f = wrapper(f).inner`, and the rest of your code is run accoridanly.  

we use `functools.wraps` in order to preserve info of the wrapped function on outside calls.

In [ ]:
def args_wrapper(*wrapper_args, **wrapper_kwargs):
    print(wrapper_args, wrapper_kwargs)
    return wrapper

@args_wrapper(1, a=2)
def f():
    print('hi')
f()

usually we'll define wrapper inside args_wrapper so it'll have access to `wrapper_args`, `wrapper_kwargs`

In [ ]:
def option_kw_wrapper(func=None, *, a=5, b=6):
    print(a,b)
    if func is None:
        return wrapper
    else:
        return wrapper(func)


@option_kw_wrapper
def f():
    print('hi f')
f()

@option_kw_wrapper(b=1, a=2)
def g():
    print('hi g')
g()

## Class Decorators

### Basic class decorator

this class reserve all attributes of its parent (some immutable class you want your function to be an instance of), except calling it is like calling fun.

we use `update_wrapper` because `wraps` is only for function decorators. both are doing the same thing.

In [ ]:
class Example:
    def __init__(self, function, *args, **kwargs):
        super().__init__(*args, **kwargs)
        functools.update_wrapper(self, function)
        self._function = function
    
    def __call__(self, *args, **kwargs):
        return self._function(*args, **kwargs)

@Example
def fun(a,b):
    print(a,b)

### class with __init__ arguments

In [ ]:
class Example:
    def __init__(self, function, arg1, arg2):
        functools.update_wrapper(self, function)
        self._function = function
        self._arg1 = arg1
        self._arg2 = arg2
    
    def __call__(self, *arg, **kwargs):
        print(self._arg1, self._arg2)
        return self._function(*arg, **kwargs)

def example_wrapper(*args, **kwargs):
    def wrapper(function):
        return Example(function, *args, **kwargs)
    return wrapper

@example_wrapper(1, 3)
def fun(a, b):
    print(a, b)

fun(4,5)

### More info

In [ ]:
# decorating a class is equivalent to `a = wrapper(ExampleClass)`
@wrapper
class A:
    def __init__(self):
        print('init')
a = A()

In [ ]:
# nesting decorators are executed top to bottom
@args_wrapper(3, d=5)
@wrapper
def f():
    print('hi')
f()